In [1]:
from __future__ import division
import tensorflow as tf
import os
import pandas as pd
import numpy as np
from nets import inception_utils, inception_v3
from glob import glob
import imageio
import sys
from sklearn.utils import shuffle
from sklearn.preprocessing import OneHotEncoder

/Users/gaoqitong/anaconda/envs/py27/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [7]:
def get_checkpoint_init_fn():
    # Load from .ckpt file
    variables_to_restore = slim.get_variables_to_restore(exclude=["InceptionV3/Logits/Conv2d_1c_1x1/weights:0", "InceptionV3/Logits/Conv2d_1c_1x1/biases:0"])
    global_step_reset = tf.assign(tf.train.get_or_create_global_step(), 0)
    slim_init_fn = slim.assign_from_checkpoint_fn("./inception_v3.ckpt",variables_to_restore,ignore_missing_vars=True)
    return slim_init_fn

In [8]:
# Load Data
if sys.platform == "darwin":
    base_skin_dir = os.path.join('./Data/')
else:
    base_skin_dir = os.path.join('/datacommons/plusds/skin_cancer/team2')
slim = tf.contrib.slim

# Set Params
MAX_EPOCH = 30000
NUM_CLASSES = 7
NUM_IMG_FROM_EACH_CLASS = 9
input_size = NUM_IMG_FROM_EACH_CLASS * NUM_CLASSES

session_config = tf.ConfigProto(log_device_placement=False)
session_config.gpu_options.allow_growth = True

In [9]:
# Dictionary for Loading Resized Images
imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x
                     for x in glob(os.path.join(base_skin_dir, 'HAM10000_images_part_[1-2]_resize', '*.jpg'))}

lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'dermatofibroma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}

# Side Information DataFrame

side_df = pd.read_csv(os.path.join(base_skin_dir, 'HAM10000_metadata.csv'))
side_df['path'] = side_df['image_id'].map(imageid_path_dict.get)
side_df['cell_type'] = side_df['dx'].map(lesion_type_dict.get) 
side_df['cell_type_idx'] = pd.Categorical(side_df['cell_type']).codes

image_by_type = [side_df.iloc[np.array(side_df["cell_type_idx"] == i)] for i in range(len(lesion_type_dict))]

In [21]:
g = tf.Graph()

with g.as_default():
#     img_holder = tf.placeholder(shape=[input_size,299,299,3], dtype=tf.float32, name="Img_Holder")
#     label_holder = tf.placeholder(shape=[input_size,7], dtype=tf.float32, name="Label_Holder")
    img = tf.Variable(tf.zeros([input_size,299,299,3]), name="Img_Var")
    label = tf.Variable(tf.zeros([input_size,7]), name="Label_Var")
#     dataset = tf.data.Dataset.from_tensor_slices((img_holder,label_holder)).batch(input_size)
#     iterator = dataset.make_initializable_iterator()
#     image, label = iterator.get_next()
    with slim.arg_scope(inception_v3.inception_v3_arg_scope()):
        logits, end_points = inception_v3.inception_v3(img, num_classes=7, create_aux_logits=False)
    loss = tf.losses.softmax_cross_entropy(label, logits)
    learning_rate = tf.train.exponential_decay(1e-04, tf.train.get_or_create_global_step(), input_size / 64 * 2.5 , 0.94)
    opt = tf.train.GradientDescentOptimizer(learning_rate)
    train_tensor = slim.learning.create_train_op(loss, optimizer=opt)
    # Creat Summary
    slim.summaries.add_scalar_summary(loss, 'cross_entropy_loss', 'losses')
    slim.summaries.add_scalar_summary(learning_rate, 'learning_rate', 'training')
    
#     variables_to_restore = slim.get_variables_to_restore(exclude=["InceptionV3/Logits/Conv2d_1c_1x1/weights:0", "InceptionV3/Logits/Conv2d_1c_1x1/biases:0"])
#         #         global_step_reset = tf.assign(tf.train.get_or_create_global_step(), 0)
#     init_assign_op, init_feed_dict = slim.assign_from_checkpoint("./inception_v3.ckpt",variables_to_restore,ignore_missing_vars=True)
#     init_feed_dict[img_holder] = input_images
#     init_feed_dict[label_holder] = labels
#     def InitAssignFn(sess):
#         sess.run([init_assign_op, iterator.initializer], init_feed_dict)

In [17]:
# Train
one_hot_encoder = OneHotEncoder(NUM_CLASSES)
one_hot_encoder.fit(np.arange(NUM_CLASSES).reshape(-1,1))

with g.as_default():
    for epoch in range(MAX_EPOCH):

        print "Traning Epoch: ", epoch
        
        tf.add_to_collection(tf.GraphKeys.UPDATE_OPS, tf.assign())
        
        # Sample Training Batch Directly from File -- These steps need to be done within the slim.learning.train() function
#         input_path = np.array([image_by_type[i]["path"].sample(NUM_IMG_FROM_EACH_CLASS) for i in range(NUM_CLASSES)]).reshape(-1)
        input_images = np.array([imageio.imread(i) for i in np.array([image_by_type[i]["path"].sample(NUM_IMG_FROM_EACH_CLASS) for i in range(NUM_CLASSES)]).reshape(-1)]).astype(np.float32)
        input_size = np.shape(input_images)[0]
        labels = np.array([[i]*NUM_IMG_FROM_EACH_CLASS for i in range(NUM_CLASSES)]).reshape(-1,1)
        input_images, labels = shuffle(input_images, labels)
        labels = one_hot_encoder.transform(labels).toarray()
        
        

        slim.learning.train(
            train_tensor,
            "./saved_model/",
            log_every_n_steps=300,
            number_of_steps=1,
            graph=g,
            save_summaries_secs=300,
            save_interval_secs=600,
            init_fn=InitAssignFn,
            global_step=tf.train.get_global_step(),
            session_config=session_config,
            init_feed_dict={img_holder:input_images, label_holder:labels})

Traning Epoch:  0
[<tf.Variable 'InceptionV3/Conv2d_1a_3x3/weights:0' shape=(3, 3, 3, 32) dtype=float32_ref>, <tf.Variable 'InceptionV3/Conv2d_1a_3x3/BatchNorm/beta:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'InceptionV3/Conv2d_1a_3x3/BatchNorm/moving_mean:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'InceptionV3/Conv2d_1a_3x3/BatchNorm/moving_variance:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'InceptionV3/Conv2d_2a_3x3/weights:0' shape=(3, 3, 32, 32) dtype=float32_ref>, <tf.Variable 'InceptionV3/Conv2d_2a_3x3/BatchNorm/beta:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'InceptionV3/Conv2d_2a_3x3/BatchNorm/moving_mean:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'InceptionV3/Conv2d_2a_3x3/BatchNorm/moving_variance:0' shape=(32,) dtype=float32_ref>, <tf.Variable 'InceptionV3/Conv2d_2b_3x3/weights:0' shape=(3, 3, 32, 64) dtype=float32_ref>, <tf.Variable 'InceptionV3/Conv2d_2b_3x3/BatchNorm/beta:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'InceptionV3/Conv2d_2b_3x3/

<function callback at 0x12aad87d0>


NameError: name 'init_feed_dict' is not defined